In [ ]:
import numpy as np

import argparse

from collections import namedtuple

import gzip
import pickle
import time

from pathlib import Path

import torch

from model import SessionGraph, forward, trans_to_cuda, trans_to_cpu
from utils import Data

In [ ]:
filename_tagnn_input_data = './ziob_yoochoose_1_64n.pcklz'
filename_tagnn_input_data = '..\Diginetica Dataset Preparation\ziob_diginetica_n.pcklz'
filename_tagnn_input_data = '../Diginetica Dataset Preparation/ziob_diginetica_n.pcklz'
filename_tagnn_model = 'tagnn_model2.pth'
filename_tagnn_recommendations = 'recommendations.pcklz'
embeddings_dimensionality = 100
number_of_nodes = -1
batch_size = 32
training_epochs = 50

In [ ]:
print('CUDA: ', torch.cuda.is_available())

## TAGNN Model Trainer

In [ ]:
with gzip.open(filename_tagnn_input_data, 'rb') as f:
    train_data = pickle.load(f)
    test_data = pickle.load(f)
    pids = pickle.load(f)

if number_of_nodes == -1:
    number_of_nodes = len(pids)

In [ ]:
train_data = Data(train_data, shuffle=True)
# test_data = Data(test_data, shuffle=False)

TAGNN_parameters = namedtuple('TAGNN_parameters', 'hiddenSize n_node batchSize nonhybrid step lr l2 lr_dc_step lr_dc')
tagnn_params = TAGNN_parameters(embeddings_dimensionality, number_of_nodes, batch_size, True, 1, 0.001, 1e-5, 3, 0.1)

model = trans_to_cuda(SessionGraph(tagnn_params)).float()

In [ ]:
for epoch in range(training_epochs):
    t0 = time.time()

    model.train()

    losses = []
    slices = train_data.generate_batch(batch_size)
    for i, j in zip(slices, np.arange(len(slices))):
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
        print('targets: ', targets)
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        losses.append(loss.item())

    model.scheduler.step()

    print(epoch, time.time() - t0, sum(losses) / len(slices))
    torch.save(model.state_dict(), filename_tagnn_model[:-4] + ('.%03d' % epoch) + filename_tagnn_model[-4:])

In [ ]:
torch.save(model.state_dict(), filename_tagnn_model)

## TAGNN Model Evaluator

In [ ]:
def evaluate_target_positions(recommendations, targets):
    positions = np.ones(len(targets)) * recommendations.shape[1]
    for i in range(len(targets)):
        indices_ = np.where(recommendations[i, :] == targets[i])[0]
        if len(indices_) > 0:
            positions[i] = indices_[0]
    return positions

In [ ]:
with gzip.open(filename_tagnn_input_data, 'rb') as f:
    train_data = pickle.load(f)
    test_data = pickle.load(f)
    pids = pickle.load(f)

if number_of_nodes == -1:
    number_of_nodes = len(pids)

sessions = test_data[0]

# if pids is not None:
#     id2pid = dict(zip(np.arange(len(pids)), pids))
#     pid2id = {j: i for (i, j) in id2pid.items()}

In [ ]:
filename_tagnn_model = 'tagnn_model.pth'
TAGNN_parameters = namedtuple('TAGNN_parameters', 'hiddenSize n_node batchSize nonhybrid step lr l2 lr_dc_step lr_dc')
tagnn_params = TAGNN_parameters(embeddings_dimensionality, number_of_nodes, batch_size, True, 1, 0.001, 1e-5, 3, 0.1)

model = SessionGraph(tagnn_params)

model.load_state_dict(torch.load(filename_tagnn_model))
model.eval()
model = trans_to_cuda(model).float()

In [ ]:
t0 = time.time()

train_data = Data((sessions, np.zeros(len(sessions))), shuffle=False)

results = []
scores = []

with torch.no_grad():
    slices = train_data.generate_batch(batch_size)
    for i in slices:
        _, scores_ = forward(model, i, train_data)
        scores_ = scores_.cpu().detach().numpy()
        results_ = scores_.argsort(axis=1)
        results_200 = results_[:, -200:][:, ::-1]
        scores_200 = np.array([scores_[i_, results_200[i_, :]] for i_ in range(scores_.shape[0])])
        # IMPORTANT: results_ must be increased by 1, because of the TAGNN representations of product
        #            identifiers - the first product (id = 0) is considered artificial (used as an internal mark
        #            in the TAGNN code), and scores_.shape[1] = len(pids) - 1, so scores_[:, 0] corresponds to
        #            the first real product (id = 1), scores_[:, 1] to the second real product (id = 2), etc.
        results_200 = results_200 + 1
        results.append(results_200)
        scores.append(scores_200)

results = np.vstack(results)
scores = np.vstack(scores)

# if pids is not None:
#     results = np.array([[id2pid[id_] if id_ in id2pid.keys() else -1 for id_ in session] for session in results])

# with gzip.open(filename_tagnn_recommendations, 'wb') as f:
#     pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)
#     pickle.dump(scores, f, protocol=pickle.HIGHEST_PROTOCOL)

print(time.time() - t0, results.shape, scores.shape)

In [ ]:
positions = evaluate_target_positions(results, test_data[1])
print(positions.min(), positions.mean(), positions.max())
print((positions < 20).sum() / len(positions))